In [1]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
from matplotlib import colors as mcolors
from matplotlib import patches as mpatches
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess
import networkx as nx

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt
import utils

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False
OVERWRITE = False


In [2]:
# output data for analysis

users = dt.get_users()
user_weeks = dt.get_user_by_week_panel(overwrite=True)

user_weeks['can_attach_recv'] = user_weeks['week'] >= dt.as_week(globals.aw_date)
user_weeks['can_attach_send'] = user_weeks['week'] >= dt.as_week(globals.inv_date)
user_weeks['sn_is_noncustodial'] = user_weeks['week'] >= dt.as_week(globals.nc_date)
users['first_autowithdraw_week'] = dt.as_week(users['first_autowithdraw_date'])
users['first_p2p_zap_week'] = dt.as_week(users['first_p2p_zap_date'])

user_weeks = user_weeks.merge(
    users[['userId', 'first_autowithdraw_week', 'first_p2p_zap_week']],
    on='userId',
    how='left'
)

user_weeks['attached_recv_wallet'] = user_weeks['week'] >= user_weeks['first_autowithdraw_week']
user_weeks['attached_send_wallet'] = user_weeks['week'] >= user_weeks['first_p2p_zap_week']

user_weeks['log_rolling_profit1'] = user_weeks['rolling_profit1'].apply(utils.signed_log)
user_weeks['log_rolling_profit0'] = user_weeks['rolling_profit0'].apply(utils.signed_log)
user_weeks['log_rolling_posts'] = user_weeks['rolling_posts'].apply(utils.signed_log)
user_weeks['log_rolling_items'] = user_weeks['rolling_items'].apply(utils.signed_log)

mask = (~user_weeks['userId'].isin(globals.sn_employee_ids)) & \
       (user_weeks['userId'] != globals.anon_id)
user_weeks = user_weeks.loc[mask].reset_index(drop=True)

mask = (user_weeks['can_attach_recv']) & (user_weeks['week'] <= user_weeks['first_autowithdraw_week'])
user_weeks.loc[mask].to_parquet(
    os.path.join(DATA_PATH, "tempdf1.parquet"),
    index=False
)

mask = (user_weeks['can_attach_send']) & (user_weeks['week'] <= user_weeks['first_p2p_zap_week'])
user_weeks.loc[mask].to_parquet(
    os.path.join(DATA_PATH, "tempdf2.parquet"),
    index=False
)


c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:731: UserWarning: Converting to Period representation will drop timezone information.
  week_start = globals.data_start.to_period('W-SAT').start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:732: UserWarning: Converting to Period representation will drop timezone information.
  week_end = globals.data_end.to_period('W-SAT').start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:91: UserWarning: Converting to Period representa

In [3]:
user_weeks[['rolling_profit1', 'log_rolling_profit1']]

,rolling_profit1,log_rolling_profit1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1238864,NaN,NaN
1238865,NaN,NaN
1238866,NaN,NaN
1238867,NaN,NaN
